In [ ]:
!python -m pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pycsp3
!python -m pip install --upgrade pycsp3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 50.3 MB/s eta 0:00:00


In [ ]:
!java --version

openjdk 11.0.24 2024-07-16
OpenJDK Runtime Environment (build 11.0.24+8-post-Ubuntu-1ubuntu322.04)
OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Ubuntu-1ubuntu322.04, mixed mode, sharing)


In [3]:
import pycsp3
import numpy as np
import json
from pycsp3 import *

In [ ]:
clear()
unpost(ALL)

filename = "donnees_cop/celar_50_7_10_5_0.800000_8.json"

with open(filename, "r") as f:
    data = json.load(f)

nbr_stations = len(data['stations'])

tableau_freq = [[data['stations'][i]['emetteur'], data['stations'][i]['recepteur']] for i in range(nbr_stations)]

delt = [data['stations'][i]['delta'] for i in range(nbr_stations)]

delta= [[0 for i in range(nbr_stations)] for j in range(nbr_stations)]

for i in range(len(data['interferences'])):
  x = data['interferences'][i]['x']
  y = data['interferences'][i]['y']
  delta[x][y] = data['interferences'][i]['Delta']

liaison=[(data['liaisons'][i]['x'], data['liaisons'][i]['y']) for i in range(len(data['liaisons']))]


liste_regions=[[] for i in range(len(data['regions']))]
for i in range(nbr_stations):
  region = data['stations'][i]['region']
  liste_regions[region].append(data['stations'][i]['num'])


X = VarArray(size=[nbr_stations,2], dom = lambda i,j: tableau_freq[i][j])


pycsp3.satisfy(
    [(abs(X[i][0] - X[i][1]) == delt[i]) for i in range(nbr_stations)],
    [(abs(X[i][k] - X[j][l]) >= delta[i][j]) for i in range(nbr_stations) for j in range(nbr_stations) for k in range(2) for l in range(2)],
    [(X[i][0] == X[j][1]) for (i,j) in liaison],
    [(NValues(X[liste_regions[k], :])) <= data['regions'][k] for k in range(len(data['regions']))]
  )

#minimize(NValues(X))
#minimize(Maximum(X))
minimize(Sum(X))
#minimize(Maximum(X)-Minimum(X))

print(posted())
print(objective())

if solve(sols=ALL, solver=ACE) is OPTIMUM: #pas de solution avec CHOCO ?
  print('Nombre de solutions optimales: ', n_solutions())
  print(values(X))


intension(function:eq(dist(X[0][0],X[0][1]),154))
intension(function:eq(dist(X[1][0],X[1][1]),154))
intension(function:eq(dist(X[2][0],X[2][1]),154))
intension(function:eq(dist(X[3][0],X[3][1]),154))
intension(function:eq(dist(X[4][0],X[4][1]),154))
intension(function:eq(dist(X[5][0],X[5][1]),154))
intension(function:eq(dist(X[6][0],X[6][1]),154))
intension(function:eq(dist(X[7][0],X[7][1]),154))
intension(function:eq(dist(X[8][0],X[8][1]),154))
intension(function:eq(dist(X[9][0],X[9][1]),154))
intension(function:eq(dist(X[10][0],X[10][1]),154))
intension(function:eq(dist(X[11][0],X[11][1]),154))
intension(function:eq(dist(X[12][0],X[12][1]),154))
intension(function:eq(dist(X[13][0],X[13][1]),154))
intension(function:eq(dist(X[14][0],X[14][1]),154))
intension(function:eq(dist(X[15][0],X[15][1]),154))
intension(function:eq(dist(X[16][0],X[16][1]),154))
intension(function:eq(dist(X[17][0],X[17][1]),154))
intension(function:eq(dist(X[18][0],X[18][1]),154))
intension(function:eq(dist(X[19]

## 2. WCSP

In [ ]:
!pip install --upgrade pip
!pip install pytoulbar2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 33.3 MB/s eta 0:00:00


In [6]:
import sys
from random import seed, randint
seed(123456789)
import pytoulbar2

N = int(sys.argv[1])

top = N**2 +1

Problem = pytoulbar2.CFN(top)

for i in range(N):
    Problem.AddVariable('Q' + str(i+1), ['row' + str(a+1) for a in range(N)])

for i in range(N):
	for j in range(i+1,N):

		#Two queens cannot be on the same row constraints
		ListConstraintsRow = []
		for a in range(N):
			for b in range(N):
				if a != b :
					ListConstraintsRow.append(0)
				else:
				 	ListConstraintsRow.append(top)
		Problem.AddFunction([i, j], ListConstraintsRow)

		#Two queens cannot be on the same upper diagonal constraints
		ListConstraintsUpperD = []
		for a in range(N):
			for b in range(N):
				if a + i != b + j :
					ListConstraintsUpperD.append(0)
				else:
				 	ListConstraintsUpperD.append(top)
		Problem.AddFunction([i, j], ListConstraintsUpperD)

		#Two queens cannot be on the same lower diagonal constraints
		ListConstraintsLowerD = []
		for a in range(N):
			for b in range(N):
				if a - i != b - j :
					ListConstraintsLowerD.append(0)
				else:
				 	ListConstraintsLowerD.append(top)
		Problem.AddFunction([i, j], ListConstraintsLowerD)

#Random unary costs
for i in range(N):
	ListConstraintsUnaryC = []
	for j in range(N):
		ListConstraintsUnaryC.append(randint(1,N))
	Problem.AddFunction([i], ListConstraintsUnaryC)


#Problem.Dump('WeightQueen.cfn')
Problem.CFN.timer(300)
res = Problem.Solve(showSolutions = 3)
if res:
	for i in range(N):
		row = ['X' if res[0][j]==i else ' ' for j in range(N)]
		print(row)
	# and its cost
	print("Cost:", int(res[1]))


ModuleNotFoundError: No module named 'pytoulbar2'